In [1]:
#1 Danny Severe

import os
import pandas as pd
import psycopg2 as psy
import warnings

conn = psy.connect(database='loftsvc', \
                   user='dannysevere', \
                   password='Severe21!', \
                   host='dbsevere.ct4e9f3blg2v.us-west-1.rds.amazonaws.com', \
                   port='5432')

cursor = conn.cursor()

In [2]:
#2

query = ('''SELECT exceptid, record, fixstamp
            FROM exceptionlog
            WHERE fixstamp is NULL
            ; ''')
df = pd.read_sql(query, conn, index_col = 'exceptid')
pd.set_option('display.max_colwidth', 150)

print(df)

                                                                                                                       record  \
exceptid                                                                                                                        
1                    Elizabeth Marsh||Gettysburg|2024-01-17|721|Faucet dripping, dripping forever, dripping highly, morosely.   
2                                                Amanda||Hudson|2024-01-17||Mighty cold sucking from thermostat; @#$%*&! fan.   
3                                                                                Tracey Thompson|doors|Hudson|2024-01-18|893|   
4                                              Rhonda Carter|hvac||2024-01-18|268|Aircon sounds like a ghost lives inside it.   
5                                           Willie Greene||Charleston|2024-01-19|331|Internet running at 270 bits per second.   
6                     Teri Perry|windows|Detroit|2024-01-20|486|Stuck and injured @#$%*&!, lock @

/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/157076184.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, index_col = 'exceptid')


In [3]:
#3

print('Rhonda Carter has a null value where there should be the building name. To fix this we can query the tenant and building table to find the building id and name and then plug it into spot it should go on the exceptionlog\n')
print('Willie Green has a null value in the type column. To fix this, we could manually read the discription and infer the type and fix it. Or we could just input other for any null value found.\n')
print('Jerry Massey has a null in the date column. To fix this we can check the dates of the entries before and after to determine if it was recorded on the same date or if it falls between the dates, assume the same day as the earliest record to avoid further delays. Record the timestamp.\n')
print('Gabriela Carter has outlets when it should say electrical. It would be difficult to fix this algorithmically without specifically defining error handling for the term outlets. If theres an unrecognized value we will just input other.\n')

Rhonda Carter has a null value where there should be the building name. To fix this we can query the tenant and building table to find the building id and name and then plug it into spot it should go on the exceptionlog

Willie Green has a null value in the type column. To fix this, we could manually read the discription and infer the type and fix it. Or we could just input other for any null value found.

Jerry Massey has a null in the date column. To fix this we can check the dates of the entries before and after to determine if it was recorded on the same date or if it falls between the dates, assume the same day as the earliest record to avoid further delays. Record the timestamp.

Gabriela Carter has outlets when it should say electrical. It would be difficult to fix this algorithmically without specifically defining error handling for the term outlets. If theres an unrecognized value we will just input other.



In [4]:
#4

query = '''ALTER TABLE servreq
            ADD COLUMN IF NOT EXISTS exceptid INT;'''

cursor.execute(query)
conn.commit()

query = ('''SELECT srid, exceptid
            FROM servreq
            LIMIT 3
            ; ''')
df = pd.read_sql(query, conn)


print(df)

   srid exceptid
0     1     None
1     2     None
2     3     None


/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/47353628.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [5]:
#5
from datetime import datetime

query = ('''SELECT exceptid, record, fixstamp
            FROM exceptionlog
            WHERE fixstamp is NULL
            ; ''')
df = pd.read_sql(query, conn)
pd.set_option('display.max_colwidth', 150)

for index, row in df.iterrows():
    
    # set variables
    record = row['record']
    exid = row['exceptid']
    x = record.strip().split('|')
    print(x)
    name = x[0]
    type_ = x[1]
    bld = x[2]
    current_yr = datetime.now().year
    if x[3] != '':
        date = x[3]
        date = datetime.strptime(date, "%Y-%m-%d")
        date = date.date()
        date_yr = date.year
        if date_yr == current_yr:
            old_date = date
    unit = x[4]
    desc = x[5]
    
    try: 
        parts = name.split()
        fname = parts[0]
        lname = parts[1]
    
    except:
        
        if name == '':
            print('Cant Fix, No Name')
            x[0] = name
        else:
            print('Cant Fix, One Name:', name)
            x[0] = name
    query = '''
                SELECT COUNT(*)
                FROM tenant
                WHERE fname = %s AND lname = %s;
                '''

    cursor.execute(query, (fname, lname))
    y = cursor.fetchone()

    if y[0] == True:
        query = '''
                SELECT tenantid
                FROM tenant
                WHERE fname = '%s' and lname = '%s'
                ;
                '''% (fname, lname)

        df2 = pd.read_sql(query, conn)
        tenantid = df2['tenantid'].values
        print('Update Name to:', tenantid)
        x[0] = int(tenantid[0])
        
    if y[0] == False:
        print('Cant Fix, Unknown Name:', name)
        x[0] = name

    # error handling type column
    
    if type_ == 'internet':
        query = '''
                SELECT rtid
                FROM reqtype
                WHERE typename = 'connectivity'
                ;
                '''

        df4 = pd.read_sql(query, conn)
        rtid = df4['rtid'].values
        print('Update Type to:', rtid)
        x[1] = int(rtid[0])
    elif type_ == 'outlets':
        query = '''
                SELECT rtid
                FROM reqtype
                WHERE typename = 'electrical'
                ;
                '''
        
        df5 = pd.read_sql(query, conn)
        rtid = df5['rtid'].values
        print('Update Type to:', rtid)
        x[1] = int(rtid[0])
        
    elif type_ in ('internet', 'connectivity', 'doors',
                        'electrical', 'hvac', 'plumbing', 'windows', 'other'):
        query = '''
                SELECT rtid
                FROM reqtype
                WHERE typename = '%s'
                ;
                '''% (type_)
        
        df5 = pd.read_sql(query, conn)
        rtid = df5['rtid'].values
        print('Update Type to:', rtid)
        x[1] = int(rtid[0])
        
    elif type_ not in ('internet', 'connectivity', 'doors',
                        'electrical', 'hvac', 'plumbing', 'windows', 'other'):
        print('Cant Fix, Unknown Type:', type_)
        x[1] = type_
    # handling building
    
    if unit == '':
        print('Cant Fix, No Unit Number:')
        x[2] = bld
        x[4] = unit
    elif not (101 <= int(unit) <= 898):
        print('Cant Fix, Bad Unit Number:', unit)
        x[2] = bld
        x[4] = int(unit)
    elif bld not in ('Antietam', 'Bull Run', 'Charleston', 'Detroit', 'Elkins Ferry', 'Fredericksburg', 'Gettysburg', 'Hudson'):
        
        query = '''
            SELECT bldgname, bldgid
            FROM building
            WHERE CAST(SUBSTR(unitrange, 1, POSITION('-' IN unitrange) - 1) AS INTEGER) <= %s
              AND CAST(SUBSTR(unitrange, POSITION('-' IN unitrange) + 1) AS INTEGER) >= %s;
        '''

        df6 = pd.read_sql(query, conn, params=(unit, unit))
        bldid = df6['bldgid'].values
        print('Update bld to:', bldid)
        x[2] = int(bldid[0])
        x[4] = int(unit)
        
    elif bld in ('Antietam', 'Bull Run', 'Charleston', 'Detroit', 'Elkins Ferry', 'Fredericksburg', 'Gettysburg', 'Hudson'):
        query = '''
            SELECT bldgname, bldgid
            FROM building
            WHERE CAST(SUBSTR(unitrange, 1, POSITION('-' IN unitrange) - 1) AS INTEGER) <= %s
              AND CAST(SUBSTR(unitrange, POSITION('-' IN unitrange) + 1) AS INTEGER) >= %s;
        '''

        df6 = pd.read_sql(query, conn, params=(unit, unit))
        bldid = df6['bldgid'].values
        print('Update bld to:', bldid)
        x[2] = int(bldid[0])
        x[4] = int(unit)
    # handling date
    
    if x[3] == '':
        if index == 0:
            print("Can’t Fix, (No Date):")
        print('Update Date to:', date)
        x[3] = date
    
    elif date_yr != current_yr:
        date = old_date
        print('Update Date to:', date)
        x[3] = date
    if desc == '':
        print('Cant Fix, No Description')
        x[5] = desc
    
    if unit == '':
        x[4] = unit
    else:
        x[4] = int(unit)
        
    update = [exid, x[0],x[1],x[2],x[3],x[4],x[5]]
    print('Updated Record:', update, '\n')

    
    

/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/3452939378.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


['Elizabeth Marsh', '', 'Gettysburg', '2024-01-17', '721', 'Faucet dripping, dripping forever, dripping highly, morosely.']


/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/3452939378.py:62: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(query, conn)
/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/3452939378.py:149: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6 = pd.read_sql(query, conn, params=(unit, unit))


Update Name to: [2]
Cant Fix, Unknown Type: 
Update bld to: [7]
Updated Record: [1, 2, '', 7, '2024-01-17', 721, 'Faucet dripping, dripping forever, dripping highly, morosely.'] 

['Amanda', '', 'Hudson', '2024-01-17', '', 'Mighty cold sucking from thermostat; @#$%*&! fan.']
Cant Fix, One Name: Amanda
Cant Fix, Unknown Name: Amanda
Cant Fix, Unknown Type: 
Cant Fix, No Unit Number:
Updated Record: [2, 'Amanda', '', 'Hudson', '2024-01-17', '', 'Mighty cold sucking from thermostat; @#$%*&! fan.'] 

['Tracey Thompson', 'doors', 'Hudson', '2024-01-18', '893', '']


/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/3452939378.py:107: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5 = pd.read_sql(query, conn)


Update Name to: [694]
Update Type to: [2]
Update bld to: [8]
Cant Fix, No Description
Updated Record: [3, 694, 2, 8, '2024-01-18', 893, ''] 

['Rhonda Carter', 'hvac', '', '2024-01-18', '268', 'Aircon sounds like a ghost lives inside it.']
Update Name to: [455]
Update Type to: [5]
Update bld to: [2]
Updated Record: [4, 455, 5, 2, '2024-01-18', 268, 'Aircon sounds like a ghost lives inside it.'] 

['Willie Greene', '', 'Charleston', '2024-01-19', '331', 'Internet running at 270 bits per second.']
Update Name to: [593]
Cant Fix, Unknown Type: 


/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/3452939378.py:135: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6 = pd.read_sql(query, conn, params=(unit, unit))


Update bld to: [3]
Updated Record: [5, 593, '', 3, '2024-01-19', 331, 'Internet running at 270 bits per second.'] 

['Teri Perry', 'windows', 'Detroit', '2024-01-20', '486', 'Stuck and injured @#$%*&!, lock @#$%*&! handle will not open.']
Cant Fix, Unknown Name: Teri Perry
Update Type to: [8]
Update bld to: [4]
Updated Record: [6, 'Teri Perry', 8, 4, '2024-01-20', 486, 'Stuck and injured @#$%*&!, lock @#$%*&! handle will not open.'] 

['Victoria Ward', 'doors', '', '2024-01-21', '345', 'Door bashed with an axe bashed with an axe trapped locked inside.']
Update Name to: [504]
Update Type to: [2]
Update bld to: [3]
Updated Record: [7, 504, 2, 3, '2024-01-21', 345, 'Door bashed with an axe bashed with an axe trapped locked inside.'] 

['Jerry Massey', 'plumbing', 'Detroit', '', '431', 'No hot water. 11 days. Hope dwindling.']
Update Name to: [255]
Update Type to: [6]
Update bld to: [4]
Update Date to: 2024-01-21
Updated Record: [8, 255, 6, 4, datetime.date(2024, 1, 21), 431, 'No hot water

/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/3452939378.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5 = pd.read_sql(query, conn)


In [6]:
#6
from datetime import datetime

query = ('''SELECT exceptid, record, fixstamp
            FROM exceptionlog
            WHERE fixstamp is NULL
            ; ''')
df = pd.read_sql(query, conn)
pd.set_option('display.max_colwidth', 150)

for index, row in df.iterrows():
    
    # set variables
    record = row['record']
    exid = row['exceptid']
    x = record.strip().split('|')
    name = x[0]
    type_ = x[1]
    bld = x[2]
    current_yr = datetime.now().year
    if x[3] != '':
        date = x[3]
        date = datetime.strptime(date, "%Y-%m-%d")
        date = date.date()
        date_yr = date.year
        if date_yr == current_yr:
            old_date = date
    unit = x[4]
    desc = x[5]
    nofix = 0
    
    try: 
        parts = name.split()
        fname = parts[0]
        lname = parts[1]
    
    except:
        
        if name == '':
            
            x[0] = name
            nofix += 1
        else:
            
            x[0] = name
            nofix += 1
    query = '''
                SELECT COUNT(*)
                FROM tenant
                WHERE fname = %s AND lname = %s;
                '''

    cursor.execute(query, (fname, lname))
    y = cursor.fetchone()

    if y[0] == True:
        query = '''
                SELECT tenantid
                FROM tenant
                WHERE fname = '%s' and lname = '%s'
                ;
                '''% (fname, lname)

        df2 = pd.read_sql(query, conn)
        tenantid = df2['tenantid'].values
        x[0] = int(tenantid[0])
        
    if y[0] == False:
        
        x[0] = name
        nofix += 1
    # error handling type column
    
    if type_ == 'internet':
        query = '''
                SELECT rtid
                FROM reqtype
                WHERE typename = 'connectivity'
                ;
                '''

        df4 = pd.read_sql(query, conn)
        rtid = df4['rtid'].values
        x[1] = int(rtid[0])
    elif type_ == 'outlets':
        query = '''
                SELECT rtid
                FROM reqtype
                WHERE typename = 'electrical'
                ;
                '''
        
        df5 = pd.read_sql(query, conn)
        rtid = df5['rtid'].values
        x[1] = int(rtid[0])
    
    elif type_ in ('internet', 'connectivity', 'doors',
                        'electrical', 'hvac', 'plumbing', 'windows', 'other'):
        query = '''
                SELECT rtid
                FROM reqtype
                WHERE typename = '%s'
                ;
                '''% (type_)
        
        df5 = pd.read_sql(query, conn)
        rtid = df5['rtid'].values
        x[1] = int(rtid[0])
        
    elif type_ not in ('internet', 'connectivity', 'doors',
                        'electrical', 'hvac', 'plumbing', 'windows', 'other'):
        
        x[1] = type_
        nofix += 1
    # handling building
    
    if unit == '':
        
        x[2] = bld
        x[4] = unit
        nofix += 1
    elif not (101 <= int(unit) <= 898):
        
        x[2] = bld
        x[4] = int(unit)
        nofix += 1
    elif bld not in ('Antietam', 'Bull Run', 'Charleston', 'Detroit', 'Elkins Ferry', 'Fredericksburg', 'Gettysburg', 'Hudson'):
        
        query = '''
            SELECT bldgname, bldgid
            FROM building
            WHERE CAST(SUBSTR(unitrange, 1, POSITION('-' IN unitrange) - 1) AS INTEGER) <= %s
              AND CAST(SUBSTR(unitrange, POSITION('-' IN unitrange) + 1) AS INTEGER) >= %s;
        '''

        df6 = pd.read_sql(query, conn, params=(unit, unit))
        bldid = df6['bldgid'].values
        x[2] = int(bldid[0])
        x[4] = int(unit)
        
    elif bld in ('Antietam', 'Bull Run', 'Charleston', 'Detroit', 'Elkins Ferry', 'Fredericksburg', 'Gettysburg', 'Hudson'):
        query = '''
            SELECT bldgname, bldgid
            FROM building
            WHERE CAST(SUBSTR(unitrange, 1, POSITION('-' IN unitrange) - 1) AS INTEGER) <= %s
              AND CAST(SUBSTR(unitrange, POSITION('-' IN unitrange) + 1) AS INTEGER) >= %s;
        '''

        df6 = pd.read_sql(query, conn, params=(unit, unit))
        bldid = df6['bldgid'].values
        x[2] = int(bldid[0])
        x[4] = int(unit)
    # handling date
    
    if x[3] == '':
        x[3] = date
        nofix += 1
    
    elif date_yr != current_yr:
        date = old_date
        x[3] = date
    if desc == '':
        
        x[5] = desc
        nofix += 1
    if unit == '':
        x[4] = unit
    else:
        x[4] = int(unit)
        
    update = [exid, x[0],x[1],x[2],x[3],x[4],x[5]]
    
    if nofix == 0:
        print('Ingesting Record into ServReq')
        cursor.execute('''INSERT INTO ServReq (tenantid, rtid, bldgid, reqdate, unit, reqdescription, compstamp, exceptid)
                                        VALUES (%d, %d, %d, '%s', %d, '%s', current_timestamp, '%s')'''% (x[0], x[1], x[2], x[3], x[4], x[5], exid))
        conn.commit()
        print('Ingestion Complete')
        print('Record Added:', update, '\n')
        
        cursor.execute('''UPDATE ExceptionLog
                          SET fixstamp = current_timestamp
                          WHERE exceptid = '%s';
                          '''% (exid))
        conn.commit()
    else:
        print('Cant Fix:', x, '\n')
    
    nofix = 0

    
    

/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/2385387768.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Cant Fix:

/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/2385387768.py:64: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(query, conn)
/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/2385387768.py:149: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6 = pd.read_sql(query, conn, params=(unit, unit))
/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/2385387768.py:106: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5 = pd.read_sql(query, conn)


 [2, '', 7, '2024-01-17', 721, 'Faucet dripping, dripping forever, dripping highly, morosely.'] 

Cant Fix: ['Amanda', '', 'Hudson', '2024-01-17', '', 'Mighty cold sucking from thermostat; @#$%*&! fan.'] 

Cant Fix: [694, 2, 8, '2024-01-18', 893, ''] 



/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/2385387768.py:136: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6 = pd.read_sql(query, conn, params=(unit, unit))


Ingesting Record into ServReq
Ingestion Complete
Record Added: [4, 455, 5, 2, '2024-01-18', 268, 'Aircon sounds like a ghost lives inside it.'] 

Cant Fix: [593, '', 3, '2024-01-19', 331, 'Internet running at 270 bits per second.'] 

Cant Fix: ['Teri Perry', 8, 4, '2024-01-20', 486, 'Stuck and injured @#$%*&!, lock @#$%*&! handle will not open.'] 

Ingesting Record into ServReq
Ingestion Complete
Record Added: [7, 504, 2, 3, '2024-01-21', 345, 'Door bashed with an axe bashed with an axe trapped locked inside.'] 

Cant Fix: [255, 6, 4, datetime.date(2024, 1, 21), 431, 'No hot water. 11 days. Hope dwindling.'] 

Cant Fix: [517, '', 1, '2024-01-21', 105, 'Unexpected a/c incursions.'] 

Ingesting Record into ServReq
Ingestion Complete
Record Added: [10, 538, 1, 7, '2024-01-22', 753, 'Slow download of wireless, hopelessness of websites; Internet law suit.'] 

Cant Fix: ['Court Boone', 1, 'Charleston', '2024-01-22', '', 'Slow download wireless for hopeless websites for.'] 

Ingesting Record 

/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/2385387768.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5 = pd.read_sql(query, conn)


Ingesting Record into ServReq
Ingestion Complete
Record Added: [19, 610, 3, 2, '2024-01-26', 293, 'Forever sparks and danger, and yet I glow like the hallway specter.'] 

Cant Fix: [610, 2, 8, '2024-01-27', 841, 'Might might highly might.'] 



In [7]:
#7
query = ('''SELECT srid, tenantid, rtid, bldgid, reqdate, unit, exceptid
            FROM ServReq
            WHERE exceptid IS NOT NULL
            ; ''')
df = pd.read_sql(query, conn)
print(df)

   srid  tenantid  rtid  bldgid     reqdate  unit  exceptid
0   115       455     5       2  2024-01-18   268         4
1   116       504     2       3  2024-01-21   345         7
2   117       538     1       7  2024-01-22   753        10
3   118       116     2       1  2024-01-23   176        12
4   119       495     3       4  2024-01-25   413        14
5   120       610     3       2  2024-01-26   293        19


/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/1698317933.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [8]:
#8
query = ('''SELECT exceptid, record, fixstamp
            FROM ExceptionLog
            WHERE fixstamp IS NOT NULL
            ; ''')
df = pd.read_sql(query, conn)
print(df)

   exceptid  \
0         4   
1         7   
2        10   
3        12   
4        14   
5        19   

                                                                                                                record  \
0                                       Rhonda Carter|hvac||2024-01-18|268|Aircon sounds like a ghost lives inside it.   
1                Victoria Ward|doors||2024-01-21|345|Door bashed with an axe bashed with an axe trapped locked inside.   
2   Shannon Ramos|connectivity||2024-01-22|753|Slow download of wireless, hopelessness of websites; Internet law suit.   
3                    Darryl Vazquez|doors|Antietam|2024-01-23|176|Door wont shut, door refuses the shutting, the door.   
4                                       Laurie Lopez|electrical||2024-01-25|413|Hopeless light switches demean us all.   
5  Gabriela Carter|outlets|Bull Run|2024-01-26|293|Forever sparks and danger, and yet I glow like the hallway specter.   

                    fixstamp  
0 2024-0

/var/folders/_l/8zp3fvh91_z1q1rtm5vjpht80000gn/T/ipykernel_39460/1576968534.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
